In [5]:
from pdfquery import PDFQuery
from pymupdf import open, Page
from pymupdf.table import TableFinder
import re
from pandas import DataFrame, concat

In [6]:
doc = open("resultado1570777439.pdf")

In [13]:
def eliminate_junk_and_rename_cols(tab: TableFinder) -> DataFrame:
    df = tab.to_pandas()  # convert to pandas DataFrame
    num_col = df.iloc[0].value_counts().iloc[0]  # verifica se ha colunas inúteis
    if num_col != 1:
        num_col += 1
    df = df.iloc[:, :-num_col].join(df.iloc[:, -1:])  # remove as colunas inúteis
    columns = df.iloc[0]  # pega a primeira linha como cabeçalho
    df = df[1:]  # remove a primeira linha
    # seta a coluna RESULTADOS, COL2, COL3 como Resultados - columns 1, 2 e 3
    dict_cols = {}
    for i, c in enumerate(columns):
        if c is not None and "\n" in c:
            ficha, data = c.split("\n")
            dict_cols[i] = {"ficha": ficha, "data": data}
    df_cols = list(df.columns)
    for key, value in dict_cols.items():
        df_cols[key] = f"RES - {value['ficha']} - {value['data']}"
    df.columns = df_cols
    
    return df

In [14]:
df_final = DataFrame()
for page in doc:  # iterate the document pages
    if isinstance(page, Page):
        tabs = page.find_tables()  # find tables in the page

        for tab in tabs:
            df = eliminate_junk_and_rename_cols(tab)  # convert to pandas DataFrame
            # df = df.apply(
            #     verifica_colunas, axis=1
            # )  # verifica se há asteriscos ou colunas na primeira linha
            df_final = concat([df_final, df])  # append to the final DataFrame

In [ ]:
from pandas import Series


def trata_colunas(cell: Series) -> Series:
    for c in cell.columns:
        if "(1)" in cell[c]:
            cell[c] = cell[c].str.replace(r"\(1\)", "", regex=True)
            cell["Extrapolou"] = True
        else:
            cell["Extrapolou"] = False
        if "*" in cell[c]:
            cell[c] = cell[c].str.replace(r"\*", "", regex=True)
            cell["Varia com idade"] = True
        else:
            cell["Varia com idade"] = False
    return cell

In [15]:
df_final

,ANALITOS,RES - 1570777439 - 24/03/2024,RES - 4430270983 - 08/01/2023,RES - 3500486158 - 11/02/2019,VALORES DE REFERÊNCIA
1,Eritrócitos,"4,64","4,28","4,21(*)","3,83 a 4,99 milhões/mm3"
2,Hemoglobina,"12,8","11,6(1)","11,6(*)(1)","11,7 a 14,9 g/dL"
3,Hematócrito,"39,8","35,8","35,2(*)","35,1 a 44,1 %"
4,Hemoglobina Corpuscular Média,"27,6(1)","27,1(1)","27,6(*)(1)","27,7 a 32,7 pg"
5,VCM,"85,8","83,6","83,6(*)","83,1 a 96,8 fL"
6,Concentração de Hemoglobina\nCorpuscular,"32,2","32,4","33,0(*)","32,0 a 35,2 g/dL"
7,RDW,"15,2(1)","15,6(1)","14,9(*)(1)","11,8 a 14,2 %"
8,Leucócitos,5.820,4.700,5.750(*),3.470 a 8.290/mm3
9,Neutrófilos,3.540,2.820,3.060(*),1.526 a 5.020/mm3
10,Eosinófilos,80,70,60(*),20 a 340/mm3
